In [44]:
import pandas as pd
# song_input = 'music_data_sample.txt'
# list_input = 'list_data_out_sample.txt'
song_input = 'Music_Data_Out.txt'
list_input = 'List_Data_Out.txt'
try:
    song = pd.read_table(song_input, delimiter="--")
    album = pd.read_table(list_input, delimiter="--")
except:
    pass

print(song.head)
print(album.head)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys


<bound method NDFrame.head of                                                song_name  \
0                             Shelly - (日剧《夏洛克：未叙之章》主题曲)   
1                                      FULLMETAL TRIGGER   
2                    Knock U down - (TBS《CDTV》12・1月份开场曲)   
3                 Don't (LIVE:2019/6/18 @Zepp DiverCity)   
4      愛のために ～for love, for a child～ - (为了爱～for love,...   
...                                                  ...   
10302                                         No Excuses   
10303                                       All the Time   
10304                                     No New Friends   
10305                                          Happy Now   
10306  CAN'T STOP THE FEELING! (Original Song From Dr...   

                             singer     song_id    album_id  rating  timestamp  
0                      DEAN FUJIOKA  1406131972  2819914042       1  130000000  
1      THE RAMPAGE from EXILE TRIBE  1408601639  2819914042       1  130000000  
2     

In [45]:
import pickle
song_dict = {}
song_out_path = "songs_dict.pkl"
for i in range(song.shape[0]):
    song_dict[song.iloc[i, 2]] = song.iloc[i, 0]
pickle.dump(song_dict, open(song_out_path,"wb"))

In [46]:
# 生成字典
album_id2name = {}
album_name2id = {}
album_id2name_out = "album_id2name.pkl"
album_name2id_out = "album_name2id.pkl"
for i in range(album.shape[0]):
    album_id2name[album.iloc[i, 1]] = album.iloc[i, 0]
    album_name2id[album.iloc[i, 0]] = album.iloc[i, 1]
pickle.dump(album_id2name, open(album_id2name_out,"wb"))
pickle.dump(album_name2id, open(album_name2id_out,"wb"))

In [47]:
song_id2name = pickle.load(open(song_out_path,'rb'))
song_id2name[1407987266]

'Knock\xa0U\xa0down - (TBS《CDTV》12・1月份开场曲)'

In [48]:
album_id2name = pickle.load(open(album_id2name_out,'rb'))
album_id2name[2750999763]

'遗世独立•旷远山谷的喧嚣摆脱'

In [49]:
album_name2id = pickle.load(open(album_name2id_out,'rb'))
album_name2id['遗世独立•旷远山谷的喧嚣摆脱']

2750999763

In [50]:
song['rating']=1
song['timestamp']=130000000

In [51]:
surprise_data = song.drop(song.columns[0:2], axis=1)
album_id = surprise_data.pop('album_id')
surprise_data.insert(0, 'album_id', album_id)
print(surprise_data)

         album_id     song_id  rating  timestamp
0      2819914042  1406131972       1  130000000
1      2819914042  1408601639       1  130000000
2      2819914042  1407987266       1  130000000
3      2819914042  1407245276       1  130000000
4      2819914042  1408605111       1  130000000
...           ...         ...     ...        ...
10302  2993015219   541687309       1  130000000
10303  2993015219  1372851726       1  130000000
10304  2993015219  1351832505       1  130000000
10305  2993015219  1294817400       1  130000000
10306  2993015219   411921855       1  130000000

[10307 rows x 4 columns]


In [52]:
list(surprise_data)

['album_id', 'song_id', 'rating', 'timestamp']

In [53]:
data_path = "music.csv"
surprise_data.to_csv(data_path, index=False, header=False)

In [54]:
from surprise import SVD, KNNBaseline
from surprise import Dataset, Reader
from surprise.model_selection import cross_validate
print("开始训练模型...")
reader = Reader(line_format='user item rating timestamp', sep=',')
data = Dataset.load_from_file(data_path, reader=reader)

algo = KNNBaseline()
perf = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5)
#输出结果
# print(algo.trainset.to_inner_uid('2750999763'))

开始训练模型...
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [55]:
current_playlist = '<心动计划>触动你心跳加速的旋律'
print("歌单名称:", current_playlist)

#歌单名称转歌单id
album_id = album_name2id[current_playlist]
print("歌单id:",album_id)

#歌单id转surprise内部user id
album_inner_id = algo.trainset.to_inner_uid(str(album_id))
print("内部id:",album_inner_id)

歌单名称: <心动计划>触动你心跳加速的旋律
歌单id: 2946956241
内部id: 75


In [18]:
#根据内部id找邻近值,返回邻近的内部user id
album_neighbors = algo.get_neighbors(album_inner_id,k=10)

#把内部userid转成歌单id,再转换成歌单名称返还
album_neighbors = (algo.trainset.to_raw_uid(inner_id) for inner_id in album_neighbors)
# for album_id in album_neighbors:
#     print(album_id)
#     print(album_id2name[int(album_id)])
album_neighbors = (album_id2name[int(album_id)] for album_id in album_neighbors)

print("和歌单 《", current_playlist, "》 最相似的10个歌单为：\n")
for album in album_neighbors:
    print(album, album_name2id[album])
#     print("内部uid:", algo.trainset.to_inner_uid(str(album_name2id[album])))

和歌单 《 <心动计划>触动你心跳加速的旋律 》 最接近的10个歌单为：

迷 人 嗓 音 2882042971
♡SloMo宝藏BGM♡ 2892787946
史诗级节奏游戏战歌 2901390761
热播电音旋律【精选】持续更新ღ℡ 2938253157
有事上车再说. 2881720365
王者荣耀孙悟空战歌 （超燃）斜阳身法猴 2951679876
[电音]纯音乐，动感强[建议耳机播放] 2951172488
欧美精选|迷幻慵懒 飘渺轻灵 2984782686
怦然心动 | 宝藏旋律与心动节奏的完美邂逅 2908828253
星河滚烫，你是人间理想 3014248800
